In [1]:
import sys
from pathlib import Path

# Добавляем корневую папку проекта в sys.path
project_root = Path.cwd().parent  # Путь к корневой директории Depth_Maker
sys.path.append(str(project_root))
print(project_root)

/
